In [1]:
 !pip install botometer

In [2]:
# Upload config.py with API keys
from google.colab import files
uploaded = files.upload()

Saving config.py to config.py


In [7]:
import os
from config import KaggleKey
os.environ['KAGGLE_USERNAME'] = KaggleKey.kaggle_username
os.environ['KAGGLE_KEY'] = KaggleKey.kaggle_key
!kaggle datasets download -d albrace/twitter-data

 99% 295M/298M [00:10<00:00, 25.6MB/s]
100% 298M/298M [00:10<00:00, 30.6MB/s]


In [8]:
!unzip /content/twitter-data.zip -d /content/data

Archive:  /content/twitter-data.zip
  inflating: /content/data/clean_twitter_data/all_users.csv  
  inflating: /content/data/clean_twitter_data/pol_actor_users.csv  
  inflating: /content/data/ge2017_cand_data/ge2017_cand_data.csv  
  inflating: /content/data/twitter_data/all_twitter_data.csv  
  inflating: /content/data/twitter_data/political_twitter_data.csv  


In [9]:
import tweepy
from tweepy import OAuthHandler, Stream, StreamListener
import botometer
import pandas as pd
from config import TwitterKeys, RapidApiKey
import pprint

In [10]:
consumer_key = TwitterKeys.consumer_key
consumer_secret = TwitterKeys.consumer_secret
access_token = TwitterKeys.access_token
access_token_secret = TwitterKeys.access_token_secret

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
twitter_api = tweepy.API(auth,  wait_on_rate_limit=True)

twitter_app_auth = {
    'consumer_key': consumer_key,
    'consumer_secret': consumer_secret,
    'access_token': access_token,
    'access_token_secret': access_token_secret,
  }

rapidapi_key = RapidApiKey.rapidapi_key

bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

In [11]:
all_user_df = pd.read_csv('/content/data/clean_twitter_data/all_users.csv', index_col=0)
pol_user_df = pd.read_csv('/content/data/clean_twitter_data/pol_actor_users.csv', index_col=0)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (18,19,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
consumer_key = TwitterKeys.consumer_key
consumer_secret = TwitterKeys.consumer_secret
access_token = TwitterKeys.access_token
access_token_secret = TwitterKeys.access_token_secret

rapidapi_key = RapidApiKey.rapidapi_key

In [13]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
twitter_api = tweepy.API(auth,  wait_on_rate_limit=True)

twitter_app_auth = {
    'consumer_key': consumer_key,
    'consumer_secret': consumer_secret,
    'access_token': access_token,
    'access_token_secret': access_token_secret,
  }
bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

pp = pprint.PrettyPrinter(indent=4)


In [14]:
cond = all_user_df['user_username'].isin(pol_user_df['user_username'])
all_user_df.drop(all_user_df[cond].index, inplace=True)

In [29]:
user_freq = all_user_df['user_username'].value_counts()
user_list = []
for user in user_freq.index:
  user_list.append(user)

In [31]:
len(user_list)

4435

In [15]:
# Check a single account by screen name
result = bom.check_account('@Hombo_')
print(result['raw_scores']['english']['overall'], result['user']['user_data']['screen_name'])


1.0 Hombo_


In [34]:
results_score = []
results_name = []

In [37]:
for screen_name, result in bom.check_accounts_in(user_list):
  try:
    name = result['user']['user_data']['screen_name']
    score = result['raw_scores']['english']['overall']
    results_name.append(name)
    results_score.append(score)
    print(score, name)
  except KeyError:
    pass


0.35 revdrcathgreen1
0.14 pippabartolotti
0.1 jazzblogz
0.09 RichardKnightGP
0.1 lesliecgroves
0.03 BeckyForrestLD
0.23 AlistairPolson
0.08 southoxongreens
0.23 AdrianSHolmes
0.15 RachelSadlerTW
0.03 markgrn4Hillgdn
0.23 DrRich4Nuneaton
0.09 DanielleGlavin
0.01 AndrewAAD
0.09 maximusdogface2
0.16 CenyddEU
0.85 NewcastleEastLD
0.83 persimew
0.1 HuwLabourLeftie
0.04 NickyShepard
0.57 Bambos_MP
0.19 carl1545
0.33 EllieChowns
0.23 paul_rowlinson
0.03 LoraineBirchall
0.07 DJerrome
0.14 TorbayGreenPart
0.01 AishaJMir
0.16 stewart_berwick
0.04 yorksgreenmarti
0.03 BrianMathew4NW
0.06 dougmswanney
0.06 4TheGreenBelt
0.23 RobertLibDem
0.02 JackEEaston
0.24 rowlem
0.22 SayersGreen
0.23 VoteAndyBrown
0.09 TimLordRenew
0.33 chrisjsuk
0.19 havantgreens
0.02 rachaelashah
0.11 DrChrisJones
0.19 IndependentHawk
0.13 DrSueR
0.29 GreensWallingfd
0.04 MicalmusRolfe
0.32 CheltKeithLab
0.16 andrew4bury
0.04 TomHailwood_
0.02 amanda_jayne4
0.02 AdamVanChelt
0.29 CheshamAmersham
0.36 GreenJamesRob
0.07 Howar

In [38]:
bot_score_df = pd.DataFrame(list(zip(results_name, results_score)), columns=['result_name', 'result_score'])

In [39]:
bot_score_df

,result_name,result_score
0,revdrcathgreen1,0.35
1,pippabartolotti,0.14
2,jazzblogz,0.10
3,RichardKnightGP,0.09
4,lesliecgroves,0.10
...,...,...
4501,groomziie,0.02
4502,CharHolVicSophx,0.00
4503,stevesage,0.00
4504,rhysjos,0.02


In [40]:
bot_score_df.to_csv('bot_score.csv')

In [ ]:
for screen_name, result in bom.check_accounts_in(followers):
  print(result)